In [1]:
from collections import namedtuple
import scipy.optimize
import numpy as np

In [2]:
Food = namedtuple('Food', ['name', 'fat', 'carbs', 'protein', 'sodium', 'fiber', 'range', 'units'])

In [29]:
target_cals = 1600
target_macros = {'fat': 30, 'carbs': 25, 'protein': 45}  # fat loss
# target_macros = {'fat': 15, 'carbs': 55, 'protein': 30}  # muscle gain
targets = {'fat': target_macros['fat'] / 9 * target_cals / 100,
           'carbs': target_macros['carbs'] / 4 * target_cals / 100,
           'protein': target_macros['protein'] / 4 * target_cals / 100}
food = [
    Food(name='soymilk', fat=1.99/243, carbs=10.01/243, protein=6/243, sodium=95/243, fiber=1/243, range=(150,150), units='grams'),  # 1 cup = 243 g
    Food(name='egg', fat=6.7, carbs=0.98, protein=6.09, sodium=88, fiber=0, range=(0,2), units='large'),  # 1 large = 61 g
    Food(name='yogurt/maple syrup', fat=0.01/170, carbs=20.41/170, protein=18.01/170, sodium=67/170, fiber=0, range=(0,2*170), units='grams yogurt'),  # 1 container = 170 g
    # Food(name='yogurt', fat=0, carbs=7/170, protein=18/170, sodium=65/170, fiber=0, range=(0,4*170), units='grams'),  # 1 container = 170 g
    # Food(name='maple syrup', fat=0.01/20, carbs=13.41/20, protein=0.01/20, sodium=2/20, fiber=0, range=(0,80), units='grams'),  # 1 tbsp = 20 g
    # Food(name='celery/peanut butter', fat=0.0017*65+18, carbs=0.0297*65+6, protein=0.0069*65+8, sodium=0.8*65+5, fiber=0.016*65+2, range=(1,3)),  # 65 g + 2 tbsp = 32 g
    Food(name='celery', fat=0.0017, carbs=0.0297, protein=0.0069, sodium=0.8, fiber=0.016, range=(65,65), units='grams'),  # 100 g
    Food(name='peanut butter', fat=9/16, carbs=3/16, protein=4/16, sodium=2.5/16, fiber=1/16, range=(0.8*32,1.2*32), units='grams'),  # 2 tbsp = 32 g
    # Food(name='almond butter', fat=17.76/16, carbs=6.02/16, protein=6.71/16, sodium=2/16, fiber=3.3/16, range=(0,0), units='grams'),  # 2 tbsp = 32 g
    # Food(name='carrot/hummus', fat=0.0024*65+5, carbs=0.0958*65+4, protein=0.0093*65+2, sodium=0.69*65+125, fiber=0.028*65+1, range=(1,3)),  # 65 g + 2 tbsp = 28 g
    Food(name='carrot', fat=0.0024, carbs=0.0958, protein=0.0093, sodium=0.69, fiber=0.028, range=(65,65), units='grams'),  # 100 g
    Food(name='hummus', fat=2.5/14, carbs=2/14, protein=1/14, sodium=62.5/14, fiber=0.5/14, range=(0.8*28,1.2*28), units='grams'),  # 2 tbsp = 28 g
    Food(name='chicken', fat=0.0357, carbs=0, protein=0.3102, sodium=0.74, fiber=0, range=(270,270), units='grams'),  # 100 g
    #Food(name='tilapia', fat=0.017, carbs=0, protein=0.2008, sodium=0.52, fiber=0, range=(0,0), units='grams'),  # 100 g
    #Food(name='salmon', fat=0.044, carbs=0, protein=0.2050, sodium=0.75, fiber=0, range=(0,0), units='grams'),  # 100 g
    Food(name='broccoli', fat=0.64/156, carbs=11.2/156, protein=3.72/156, sodium=64/156, fiber=5.2/156, range=(4*78,4*78), units='grams'),  # 1/2 cup = 78 g
    Food(name='cheese', fat=6, carbs=0, protein=7, sodium=190, fiber=0, range=(2,2), units='sticks'),  # 1 stick
    Food(name='lettuce', fat=0.0015, carbs=0.0287, protein=0.0136, sodium=0.28, fiber=0.013, range=(4*36, 4*36), units='grams'),  # 1 cup = 36 g
    Food(name='tomatoes', fat=0.002, carbs=0.0389, protein=0.0088, sodium=0.05, fiber=0.012, range=(149,149), units='grams'),  # 1 cup = 149 g
    Food(name='bell peppers', fat=0.003, carbs=0.0603, protein=0.0099, sodium=0.04, fiber=0.021, range=(75,75), units='grams'),  # 1 cup = 149 g
    Food(name='vinaigrette', fat=(1+0)/2, carbs=(0+0.1703)/2, protein=(0+0.0049)/2, sodium=(0.02+0.23)/2, fiber=0, range=(16,100), units='grams'),  # 1 tbsp = 16 g
    Food(name='protein powder', fat=1.5/22, carbs=4/22, protein=15/22, sodium=90/22, fiber=2/22, range=(0,44), units='grams'),  # 1 scoop = 22 g
]
ints = ['egg', 'cheese']

In [30]:
def foo(x, prn=False):
    y = np.zeros(5)
    for i, val in enumerate(x):
        y += val * np.array([food[i].fat, food[i].carbs, food[i].protein, food[i].fiber, food[i].sodium])
    if prn:
        cals = y[0]*9 + y[1]*4 + y[2]*4
        for i, val in enumerate(x):
            print(f'    {food[i].name}: {val} {food[i].units}')
        print(f'Total Calories = {int(cals)}, Fat = {y[0]:4.1f}, Carbs = {y[1]:5.1f}, Protein = {y[2]:5.1f}, Sodium = {int(y[4]):4d}, Fiber = {y[3]:4.1f}')
        print(f'   Target Cals = {target_cals}, Fat = {targets["fat"]:.1f}, Carbs = {targets["carbs"]:.1f}, Protein = {targets["protein"]:.1f}, Sodium < 2000, Fiber = 25.0')
        print(f'Total Macros:  Fat = {y[0]*9/cals*100:4.1f}%, Carbs = {y[1]*4/cals*100:4.1f}%, Protein = {y[2]*4/cals*100:4.1f}%')
        print(f'     Targets:  Fat = {target_macros["fat"]:4.1f}%, Carbs = {target_macros["carbs"]:4.1f}%, Protein = {target_macros["protein"]:4.1f}%')
    return np.linalg.norm(np.array([targets['fat'], targets['carbs'], targets['protein'], 25]) - y[0:4])

In [31]:
def cons(x):
    return max(x[i] - int(x[i]) if food[i].name in ints else 0 for i in range(len(x)))
    #return max(x[i] - int(x[i]) for i in range(len(x)))
x0 = np.array([a.range[0] for a in food])
res = scipy.optimize.basinhopping(
    foo, x0, niter=400,
    minimizer_kwargs={'bounds': [a.range for a in food], 'constraints': ({'type': 'eq', 'fun': cons})})
# res = scipy.optimize.basinhopping(
#     foo, x0, niter=200, minimizer_kwargs={'bounds': [a.range for a in food]})

In [32]:
foo(res.x, True)

    soymilk: 150.4103622685751 grams
    egg: -0.14168954440264686 large
    yogurt/maple syrup: 263.70106109524005 grams yogurt
    celery: 65.03581831614214 grams
    peanut butter: 25.830425872024232 grams
    carrot: 64.81740664449647 grams
    hummus: 22.130004224072927 grams
    chicken: 270.2400306436817 grams
    broccoli: 311.57746760506404 grams
    cheese: 2.037124751074798 sticks
    lettuce: 144.28608510295837 grams
    tomatoes: 148.64869471807597 grams
    bell peppers: 75.0431323653052 grams
    vinaigrette: 15.769195443945268 grams
    protein powder: 44.17760720229591 grams
Total Calories = 1602, Fat = 53.8, Carbs = 100.1, Protein = 179.6, Sodium = 1298, Fiber = 25.5
   Target Cals = 1600, Fat = 53.3, Carbs = 100.0, Protein = 180.0, Sodium < 2000, Fiber = 25.0
Total Macros:  Fat = 30.2%, Carbs = 25.0%, Protein = 44.8%
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%


0.83794940782242777

In [33]:
foo(np.rint(res.x).astype(int), True)

    soymilk: 150 grams
    egg: 0 large
    yogurt/maple syrup: 264 grams yogurt
    celery: 65 grams
    peanut butter: 26 grams
    carrot: 65 grams
    hummus: 22 grams
    chicken: 270 grams
    broccoli: 312 grams
    cheese: 2 sticks
    lettuce: 144 grams
    tomatoes: 149 grams
    bell peppers: 75 grams
    vinaigrette: 16 grams
    protein powder: 44 grams
Total Calories = 1613, Fat = 54.7, Carbs = 100.3, Protein = 180.0, Sodium = 1302, Fiber = 25.5
   Target Cals = 1600, Fat = 53.3, Carbs = 100.0, Protein = 180.0, Sodium < 2000, Fiber = 25.0
Total Macros:  Fat = 30.5%, Carbs = 24.9%, Protein = 44.6%
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%


1.5077147757382126

In [34]:
for i, ent in enumerate(food):
    if ent.name == 'yogurt/maple syrup':
        yogurt = np.rint(res.x).astype(int)[i]
        maple = yogurt*20/170
print(f'{int(np.rint(maple))} grams of maple syrup')

31 grams of maple syrup
